## YOLOv3 모델 (TensorFlow)

- 코드: https://github.com/zzh8829/yolov3-tf2

### 모델 다운로드

In [ ]:
!pip uninstall -y tensorflow keras

In [ ]:
!git clone https://github.com/zzh8829/yolov3-tf2
%cd /content/yolov3-tf2
!pip install -r requirements-gpu.txt
!pip install keras==2.4.2

### 모델 파일 변환

DarkNet의 모델파일을 Keras에서 사용할 수 있는 모델 파일로 변환
- `convert.py`: 변환 실행 파일
- `yolov3.weight`: Darknet으로 학습된 모델 파일
- `yolov3.tf`: Keras Yolov3 모델


* Yolov3 Weight: https://pjreddie.com/media/files/yolov3.weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O ./data/yolov3.weights
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf

In [ ]:
# !wget --help

### Detector

In [ ]:
import time
import cv2
import numpy as np
import tensorflow as tf

from yolov3_tf2.models import YoloV3
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
from absl import app, logging, flags
from absl.flags import FLAGS

from keras.preprocessing import image
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
# def del_all_flags(FLAGS):
#     flags_dict = FLAGS._flags()
#     keys_list = [keys for keys in flags_dict]
    
#     for keys in keys_list:
#     	FLAGS.__delattr__(keys)
        
# del_all_flags(tf.flags.FLAGS)

In [ ]:
flags.DEFINE_string('classes2', './data/coco.names', 'classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf', 'weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord')
flags.DEFINE_string('output', './output.jpg', 'output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes')

app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
yolo = YoloV3(classes=FLAGS.num_classes)
yolo.load_weights(FLAGS.weights).expect_partial()
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]

In [ ]:
print(class_names)

In [ ]:
def detector():
  img_raw = tf.image.decode_image(open(FLAGS.image, 'rb').read(), channels=3)
  
  img = tf.expand_dims(img_raw, 0)
  img = transform_images(img, FLAGS.size)

  t1 = time.time()
  boxes, scores, classes, nums = yolo(img)
  t2 = time.time()
  print('time: {}'.format(t2 - t1))

  for i in range(nums[0]):
    print('{}, {}, {}'.format(class_names[int(classes[0][i])],
                              np.array(scores[0][i]),
                              np.array(boxes[0][i])))
    
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

    return img

In [ ]:
!ls data

In [ ]:
FLAGS.image = 'data/girl.png'
Image(filename=FLAGS.image, width=500)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=500))

In [ ]:
FLAGS.image = 'data/meme.jpg'
Image(filename=FLAGS.image, width=1000)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=1000))

In [ ]:
FLAGS.image = 'data/street.jpg'
Image(filename=FLAGS.image, width=1000)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=1000))

In [ ]:
FLAGS.image = 'data/meme2.jpeg'
Image(filename=FLAGS.image, width=400)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=400))